In [2]:
import pyreadr as pr
import pandas as pd
import numpy as np

In [ ]:
# note: to load datasets file path should be inserted. 

In [12]:
# Sampled individuals (ego, or focal) are identified in the file ego_individuals.RData
# In ego_individuals['handedness']: 0 = Right , 1 = Left

### Load ego_individuals.RData as 'ego' ###

ego = pr.read_r('')['Allego'] #load 'ego_individuals.RData'

print(ego.shape, "\n", ego.isna().sum(), "\n", 'handedness: \n', ego['handedness'].value_counts())
ego.head()


In [11]:
# Nurhayu et al., : "For computing the various family categories presented in Table 1 (sample's frequencies),
# a full pedigree was required, i.e. including some mock individuals.[...] The list of all added
#  mock individuals is in the mock_individuals.RData file." (Script_for_data_in_Table1)

### Find and remove mock inviduals in ego ###

mock = pr.read_r('')['mock_individuals'].values #load 'mock_individuals.RData'
mock = [x[0] for x in mock]

ego['animal'] = np.where(ego['animal'].isin(mock),np.nan ,ego['animal']) #Replace mocks in 'animal' with NaN
ego['Idfather'] = np.where(ego['Idfather'].isin(mock),np.nan ,ego['Idfather']) #Replace mocks in 'Idmother' with NaN
ego['Idmother'] = np.where(ego['Idmother'].isin(mock),np.nan ,ego['Idmother']) #Replace mocks in 'Idfather' with NaN

print(ego.shape, "\n", ego.isna().sum())


In [13]:
#  Nurhayu et al.,: "All other categories (parents, grand-parents, child,sibs, spouse and 
#  spouse's familiy) are identified topologically in the pedigrees, from ego." (Script_for_data_in_Table1)

# Runing "Script_for_data_in_Table1" creats multiple dataset for these categories. Here, 'parents' is used for 
# data on ego['animal'] parents' handedness. 

### Insert parents' handedness to ego, from parents ###

parents = pr.read_r('')['parents'] #load 'parents.RData'

# ## insert mothers' hand into ego:
# #note: some mothers apprear more than once in ego as they have several kids

moms_p = parents[parents['animal'].isin(ego['Idmother'])] # Individuals('animal') in parents who are mothers in ego
moms_p = moms_p[['animal','handedness']] # Subset these mothers' ID ('animal') and their handedness
moms_p.rename(columns = {'animal':'Idmother', 'handedness':'mother_hand' }, inplace = True) #Rename col to merge with ego

ego = pd.merge(ego,moms_p, how = 'left' , on = 'Idmother') #merge mothers' hand with ego


##insert fathers' hand into ego:
fa_p = parents[parents['animal'].isin(ego['Idfather'])] # Individuals('animal') in parents who are fathers in ego
fa_p = fa_p[['animal','handedness']] # Subset these fathers' ID ('animal') and their handedness
fa_p.rename(columns = {'animal':'Idfather', 'handedness':'father_hand' }, inplace = True) #Rename col to merge with ego

ego = pd.merge (ego,fa_p, how = 'left' , on = 'Idfather') #merge fathers' hand with ego

print(ego.shape, "\n", ego.isna().sum())
ego



In [14]:
### make ego nice and tidy ###

ego = ego.dropna(axis = 0, how = 'any') #remove rows missing values
ego = ego.reset_index() #Reset index

print(ego.shape, "\n", ego.isna().sum(), '\n', ego['mother_hand'].value_counts() ,ego['father_hand'].value_counts())
ego


In [15]:
### Fit Laland's et al (1995) analysis ###

### (1) Parents' mating ###

#note:
#In ego table: 0 = Right ; 1 = Left
# Group parents by mating: RxR = 0 , RxL or LxR = 1, LxL = 2:

ego["mating"] = np.array(range(0,len(ego['index']))) #create a new column in ego for parents mating

for i in range(0,len(ego['mating'])):
    if((ego['mother_hand'][i] == 0) and (ego['father_hand'][i] == 0)): #Both right = 0
        ego['mating'].replace(i, 0, inplace = True)
        
    if((ego['mother_hand'][i] == 1) and (ego['father_hand'][i] == 0)): # Mother is left & father is right = 1
        ego['mating'].replace(i, 1, inplace = True)
        
    if((ego['mother_hand'][i] == 0) and (ego['father_hand'][i] == 1)): # Mother is right & father is left = 1
        ego['mating'].replace(i, 1, inplace = True)
        
    if((ego['mother_hand'][i] == 1) and (ego['father_hand'][i] == 1)): # Both left = 2
        ego['mating'].replace(i, 2, inplace = True)
        
print(ego['mating'].value_counts())
ego    

In [8]:
### (2) Frequency of Right- and Left-Handed Offspring from RXR, RXL, and LxL Matings ###

# in Laland's, order is:  R|RxR  L|RxR  R|RxL  L|RxL  R|LxL  L|LxL

#note:
#In ego table: 0 = Right ; 1 = Left
# Group parents by mating: RxR = 0 , RxL or LxR = 1, LxL = 2:

hands = pd.crosstab(ego['handedness'],[ego['mating'],ego['handedness']],  margins = True, margins_name = "Toal" )
hands.drop([0,1], inplace = True)

hands


mating        0       1       2     Toal
handedness  0.0 1.0 0.0 1.0 0.0 1.0     
handedness                              
Toal        521  73  61  26   1   1  683